<a href="https://colab.research.google.com/github/NevroHelios/nn_projects/blob/main/Spam_MSG_Detection_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sms-spam-collection-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F483%2F982%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240416%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240416T142702Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D822ed2374ee3dddf4b8272f7e8dd47ffd607c8950905e3c3d2c8fe662cc1e3c3e9ccdc4946839ea60b76e80b8d6b390fece0c0643044684cc5b46bf5c9858eb0ef6700babb4b6754d09a47cbf9bb74cbea5635fbe2295ced7a42141d5bdecfa82689759a2c66d4205cbe6a5ea767245e23816c159d3e56e7246e81d59795f682977571bf06295347b324eeb2d4c8ee1b0305a25fbde318e3f49699dd0c533394df78315348e598407fcde0466662abe17075ec2a557e7eba9658fa5d46781207aee61466f4eaa548087c119d56004845b97e1a288283a2ca2a52ddec0fe9764311c1d26449ffd351f898a612165bc780d57c88f79a470289d6ca2df0fdf7adf0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 215934 bytes downloaded
Downloaded and uncompressed: sms-spam-collection-dataset
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [ ]:
import pandas as pd
import numpy as np
import re

import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='cp437')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df = df[['v1', 'v2']]
df.columns = ['target', 'text']
df.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# ham -> 1
# spam -> 0
df.loc[:, 'target'] = df['target'].apply(lambda x: 1 if x == 'ham' else 0)
df.head()

,target,text
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def pre_process(text):
    # Lowercasing
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Stopword Removal
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Punctuation Removal
    tokens = [word for word in tokens if word not in string.punctuation]

    # Stemming
    ps = PorterStemmer()
    stemmed_tokens = [ps.stem(word) for word in tokens]

    # Join tokens back into a string
    processed_text = ' '.join(stemmed_tokens)

    return processed_text

In [ ]:
abt = df.copy()
abt['Transformed_text']=abt['text'].apply(pre_process)
abt.head()

,target,text,Transformed_text
0,1,"Go until jurong point, crazy.. Available only ...",go jurong point crazi .. avail bugi n great wo...
1,1,Ok lar... Joking wif u oni...,ok lar ... joke wif u oni ...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,1,U dun say so early hor... U c already then say...,u dun say earli hor ... u c alreadi say ...
4,1,"Nah I don't think he goes to usf, he lives aro...",nah n't think goe usf live around though


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(max_features=3000)

In [ ]:
X = tfidf.fit_transform(abt['Transformed_text'])
y = abt['target'].values

In [ ]:
# X = torch.from_numpy(X.toarray())
X = X.toarray().astype('float32')
y = y.astype('float32')

X.dtype, y.dtype

(dtype('float32'), dtype('float32'))

In [ ]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

X.dtype, y.dtype, X.shape, y.shape

(torch.float32, torch.float32, torch.Size([5572, 3000]), torch.Size([5572]))

In [ ]:
X[0].shape, y[0].shape

(torch.Size([3000]), torch.Size([]))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class DetectSpamV0(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=3000, out_features=2000)
        self.layer2 = nn.Linear(in_features=2000, out_features=1000)
        self.layer3 = nn.Linear(in_features=1000, out_features=100)
        self.layer4 = nn.Linear(in_features=100, out_features=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.layer4(self.relu(self.layer3(self.relu(self.layer2(self.relu(self.layer1(x)))))))

model_0 = DetectSpamV0().to(device)
model_0

DetectSpamV0(
  (layer1): Linear(in_features=3000, out_features=2000, bias=True)
  (layer2): Linear(in_features=2000, out_features=1000, bias=True)
  (layer3): Linear(in_features=1000, out_features=100, bias=True)
  (layer4): Linear(in_features=100, out_features=1, bias=True)
  (relu): ReLU()
)

In [ ]:
# loss
loss_fn = nn.BCEWithLogitsLoss()
#optimizer
optimizer = torch.optim.Adam(params=model_0.parameters(),
                           lr=0.01)

In [ ]:
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([4457, 3000]),
 torch.Size([1115, 3000]),
 torch.Size([4457]),
 torch.Size([1115]))

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

epochs = 701

for epoch in range(epochs):
    model_0.train()

    y_logits = model_0(X_train).squeeze()
#     print(y_logits.shape)
    y_preds = torch.round(torch.sigmoid(y_logits))

    loss = loss_fn(y_logits, y_train)
    crr = torch.eq(y_train, y_preds).sum()
    acc = (crr / len(y_preds)) * 100

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if epoch % 100 == 0:
        model_0.eval()
        with torch.inference_mode():
            y_test_logits = model_0(X_test).squeeze()
            y_test_preds = torch.round(torch.sigmoid(y_test_logits))

            test_loss = loss_fn(y_test_logits, y_test)
            test_crr = torch.eq(y_test, y_test_preds).sum()
            test_acc = (test_crr / len(y_test)) * 100

            print(f"epoch {epoch} | train loss {loss:.6f} | training acc {acc:.6f}% | test loss {loss:.6f} | test acc {test_acc:.6f}%")

epoch 0 | train loss 0.714491 | training acc 13.394660% | test loss 0.714491 | test acc 86.547081%
epoch 100 | train loss 0.000000 | training acc 99.999992% | test loss 0.000000 | test acc 97.309418%
epoch 200 | train loss 0.000000 | training acc 99.999992% | test loss 0.000000 | test acc 97.309418%
epoch 300 | train loss 0.000000 | training acc 99.999992% | test loss 0.000000 | test acc 97.309418%
epoch 400 | train loss 0.000000 | training acc 99.999992% | test loss 0.000000 | test acc 97.309418%
epoch 500 | train loss 0.000000 | training acc 99.999992% | test loss 0.000000 | test acc 97.309418%
epoch 600 | train loss 0.000000 | training acc 99.999992% | test loss 0.000000 | test acc 97.309418%
epoch 700 | train loss 0.000000 | training acc 99.999992% | test loss 0.000000 | test acc 97.309418%


# Inference

In [ ]:
# below a spam msg generated from gpt

mail = """
Subject: Congratulations! You've Won a Luxury Vacation!

Dear Valued Customer,

We are delighted to inform you that you've been selected as the lucky winner of our exclusive luxury vacation giveaway! As a token of our appreciation to our loyal customers, you have won an all-expenses-paid trip to an exotic destination of your choice.

You and a guest will enjoy:

5-star accommodation at a luxurious resort
Gourmet dining experiences
Exciting activities such as snorkeling, sightseeing tours, and more
VIP treatment throughout your stay
To claim your prize, simply click on the link below and provide your contact information. Hurry, this offer is valid for a limited time only!

Claim Your Prize Now!

Please note that terms and conditions apply. By participating, you agree to receive promotional emails from our partners.

Congratulations once again on your win!

Best regards,
The Prize Patrol Team
"""

text = pre_process(mail)
text = tfidf.transform([text]).toarray().astype('float32')
text = torch.from_numpy(text)


In [ ]:
# df.loc[:, 'target'] = df['target'].apply(lambda x: 1 if x == 'ham' else 0)

infer = {0:'Spam', 1:'ham'}

with torch.inference_mode():
  logits = model_0(text.to(device)).squeeze()
  label = torch.round(torch.sigmoid(logits))

print(infer[label.item()])

Spam
